# Test for Fixed Effects

## Load Libraries and Dataset

### Libraries

In [1]:
import numpy as np
import pandas as pd

import scipy.stats as stats
import statsmodels.api as sm
from linearmodels.panel import PanelOLS

from statsmodels.iolib import load_pickle, save_pickle

### Dataset and PanelOLS Object

In [2]:
sgx = pd.read_csv("data/clean_sgx.csv", index_col= None)

In [3]:
sgx = sgx[['Company Code', 'Year', 'LEVERAGE',
           'SIZE', 'PROFITABILITY', 'TANG', 'LIQUID', 'MCAP', 'SOLV']]
sgx['year'] = pd.Categorical(sgx['Year'])
sgx['compcode'] = pd.Categorical(sgx['Company Code'])
sgx = sgx.set_index(['Company Code', 'Year'])

endo = sgx.LEVERAGE
exog_vars = ['SIZE', 'PROFITABILITY', 'TANG', 'LIQUID', 'MCAP', 'SOLV']
exog = sm.add_constant(sgx[exog_vars])

In [4]:
sgx.head(30)


LEVERAGE       SIZE  PROFITABILITY      TANG    LIQUID  \
Company Code Year                                                           
3            2015 -0.235443  17.927564      -0.094659  0.176726  1.563892   
             2016 -0.206973  17.984885      -0.010839  0.096761  1.505202   
             2017 -0.238219  18.136273      -0.013888  0.081307  1.343012   
             2018 -0.259016  18.114074       0.013371  0.079477  1.351929   
             2019 -0.222870  17.978821       0.078838  0.113450  1.558678   
             2020 -0.353373  18.184655       0.126747  0.077289  1.653512   
             2021 -0.354726  18.488321       0.118694  0.060646  1.582983   
             2022 -0.473794  18.382137       0.079766  0.063917  1.825263   
10           2015 -0.027551  17.648089       0.029971  0.094432  1.726605   
             2016  0.026441  17.779968       0.035823  0.151546  1.365213   
             2017  0.071474  17.879947       0.042540  0.163954  1.398815   
             2018  0.124074  17.825039      -0.001821  0.153358  1.303544   
             2019  0.267393  17.789708      -0.071748  0.298266  1.079697   
             2020  0.300076  17.569981       0.006588  0.283695  1.066758   
             2021  0.231850  17.314345      -0.032096  0.229591  1.047314   
             2022  0.234610  17.319097       0.068364  0.187101  1.020956   
12           2015  0.199280  16.916278      -0.037227  0.042269  0.536030   
             2016  0.073882  16.805376      -0.034099  0.039883  0.922022   
             2017  0.110147  16.475071      -0.230511  0.041917  1.021277   
             2018  0.215825  16.442061      -0.150514  0.023723  0.591639   
             2019  0.282343  16.787616       0.173040  0.056827  0.953543   
             2020  0.334869  16.820698      -0.154581  0.040020  0.921919   
             2021  0.336646  16.516268       0.065744  0.031697  0.539579   
             2022  0.256070  16.609800      -0.067886  0.033637  1.397857   
14           2015  0.226407  16.374310      -0.290508  0.111944  1.161707   
             2016  0.131923  16.313605      -0.108950  0.105254  1.003601   
             2017 -0.230777  16.558217      -0.167373  0.097032  1.170574   
             2018 -0.100940  16.666088      -0.105473  0.291681  0.815182   
             2019 -0.024855  16.548582      -0.177847  0.346863  0.558211   
             2020  0.019507  16.513552      -0.060864  0.394418  0.477831   

                           MCAP     SOLV  year compcode  
Company Code Year                                        
3            2015  3.726956e+07  -31.451  2015        3  
             2016  6.995037e+06   -7.719  2016        3  
             2017  7.919165e+06   -8.678  2017        3  
             2018  9.212378e+06    0.966  2018        3  
             2019  1.931658e+07   11.465  2019        3  
             2020  1.459661e+07   61.729  2020        3  
             2021  2.725603e+07  194.000  2021        3  
             2022  4.331235e+07   56.147  2022        3  
10           2015  1.888834e+07   -1.277  2015       10  
             2016  8.499886e+06    1.285  2016       10  
             2017  9.514661e+06    0.764  2017       10  
             2018  6.367345e+06   -2.850  2018       10  
             2019  3.209531e+06   -5.687  2019       10  
             2020  2.340626e+06   -3.112  2020       10  
             2021  1.254450e+06   -5.483  2021       10  
             2022  1.134966e+06   -0.086  2022       10  
12           2015  3.690988e+07   -2.039  2015       12  
             2016  4.643639e+07   -3.355  2016       12  
             2017  5.263948e+07  -24.447  2017       12  
             2018  3.172811e+07   -3.424  2018       12  
             2019  2.625733e+07    2.233  2019       12  
             2020  3.505423e+07   -2.289  2020       12  
             2021  2.424991e+07   -0.007  2021       12  
             2022  2.687442e+07   -4.333  2022       12  
14           2015  7.461227e+06  -53.328  2015

In [5]:
pooled_res = load_pickle("model/pooled_ols.pickle")

## Fixed Effects Estimation

In [6]:
def fixedeffect(endo = endo, exog = exog,
                entity_eff = False, time_eff = False,
                cov_type = 'unadjusted', **cov_kwargs):
    mod = PanelOLS(endo, exog,
                   entity_effects=entity_eff,
                   time_effects=time_eff)
    res = mod.fit(cov_type= cov_type, **cov_kwargs)

    return res

### Entity Fixed Effect Model

In [7]:
entity_fe_res = fixedeffect(endo, exog, entity_eff= True)
print(entity_fe_res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:               LEVERAGE   R-squared:                        0.2401
Estimator:                   PanelOLS   R-squared (Between):              0.1308
No. Observations:                1728   R-squared (Within):               0.2401
Date:                Fri, Apr 05 2024   R-squared (Overall):              0.1504
Time:                        17:25:47   Log-likelihood                    1615.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      79.286
Entities:                         216   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(6,1506)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             79.286
                            

### Time Fixed Effect Model

In [8]:
time_fe_res = fixedeffect(endo, exog, time_eff= True)
print(entity_fe_res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:               LEVERAGE   R-squared:                        0.2401
Estimator:                   PanelOLS   R-squared (Between):              0.1308
No. Observations:                1728   R-squared (Within):               0.2401
Date:                Fri, Apr 05 2024   R-squared (Overall):              0.1504
Time:                        17:25:47   Log-likelihood                    1615.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      79.286
Entities:                         216   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(6,1506)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             79.286
                            

### 2-Way Fixed Effect Model

In [9]:
tw_fe_res = fixedeffect(endo, exog, entity_eff= True, time_eff= True)
print(entity_fe_res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:               LEVERAGE   R-squared:                        0.2401
Estimator:                   PanelOLS   R-squared (Between):              0.1308
No. Observations:                1728   R-squared (Within):               0.2401
Date:                Fri, Apr 05 2024   R-squared (Overall):              0.1504
Time:                        17:25:47   Log-likelihood                    1615.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      79.286
Entities:                         216   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(6,1506)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             79.286
                            

### F-test for Fixed Effects

### Joint test for 2-Way FE

In [10]:
print(tw_fe_res.f_pooled)

Pooled F-statistic
H0: Effects are zero
Statistic: 25.3104
P-value: 0.0000
Distributed: F(222,1499)


### Conditional F-test for Entity Fixed Effects

In [11]:
print(entity_fe_res.f_pooled)

Pooled F-statistic
H0: Effects are zero
Statistic: 25.7635
P-value: 0.0000
Distributed: F(215,1506)


### Conditional F-test for Time Fixed Effects

In [12]:
print(time_fe_res.f_pooled)

Pooled F-statistic
H0: Effects are zero
Statistic: 1.0642
P-value: 0.3842
Distributed: F(7,1714)


### Marginal F-test for Entity Fixed Effects

In [13]:
exog_vars_marginal_entity = ['SIZE', 'PROFITABILITY', 'TANG', 'LIQUID', 'MCAP', 'SOLV', 'year']
exog_marginal_entity = sm.add_constant(sgx[exog_vars_marginal_entity])
print(fixedeffect(endo, exog_marginal_entity, entity_eff= True).f_pooled)

Pooled F-statistic
H0: Effects are zero
Statistic: 25.9912
P-value: 0.0000
Distributed: F(215,1499)


### Marginal F-test for Time Fixed Effects

In [14]:
exog_vars_marginal_time = ['SIZE', 'PROFITABILITY', 'TANG', 'LIQUID', 'MCAP', 'SOLV', 'compcode']
exog_marginal_time = sm.add_constant(sgx[exog_vars_marginal_time])
print(fixedeffect(endo, exog_marginal_time, time_eff = True).f_pooled)

Pooled F-statistic
H0: Effects are zero
Statistic: 3.2219
P-value: 0.0021
Distributed: F(7,1499)


Significant F-tests suggests that a 2-way fixed effects estimation model is most preferred if fixed effects are to be assumed.

### Robust Estimation of Entity fixed effects model

In [15]:
entity_clus_entity_fe = fixedeffect(endo, exog, entity_eff= True, cov_type= 'clustered', cluster_entity= True)
print(entity_clus_entity_fe.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:               LEVERAGE   R-squared:                        0.2401
Estimator:                   PanelOLS   R-squared (Between):              0.1308
No. Observations:                1728   R-squared (Within):               0.2401
Date:                Fri, Apr 05 2024   R-squared (Overall):              0.1504
Time:                        17:25:48   Log-likelihood                    1615.1
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      79.286
Entities:                         216   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(6,1506)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             18.662
                            

In [16]:
entity_clus_time_fe = fixedeffect(endo, exog, entity_eff= True, cov_type= 'clustered', cluster_time= True)
print(entity_clus_time_fe.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:               LEVERAGE   R-squared:                        0.2401
Estimator:                   PanelOLS   R-squared (Between):              0.1308
No. Observations:                1728   R-squared (Within):               0.2401
Date:                Fri, Apr 05 2024   R-squared (Overall):              0.1504
Time:                        17:25:48   Log-likelihood                    1615.1
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      79.286
Entities:                         216   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(6,1506)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             1708.8
                            

### Robust Estimation of 2-Way fixed effects model

In [17]:
tw_clus_entity_fe = fixedeffect(endo, exog, entity_eff= True, time_eff= True, cov_type= 'clustered', cluster_entity= True)
print(tw_clus_entity_fe.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:               LEVERAGE   R-squared:                        0.2294
Estimator:                   PanelOLS   R-squared (Between):              0.1387
No. Observations:                1728   R-squared (Within):               0.2397
Date:                Fri, Apr 05 2024   R-squared (Overall):              0.1569
Time:                        17:25:48   Log-likelihood                    1628.0
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      74.386
Entities:                         216   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(6,1499)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             15.283
                            

In [18]:
entity_clus_tw_fe = fixedeffect(endo, exog, entity_eff= True, time_eff= True, cov_type= 'clustered', cluster_time= True)
print(entity_clus_tw_fe.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:               LEVERAGE   R-squared:                        0.2294
Estimator:                   PanelOLS   R-squared (Between):              0.1387
No. Observations:                1728   R-squared (Within):               0.2397
Date:                Fri, Apr 05 2024   R-squared (Overall):              0.1569
Time:                        17:25:48   Log-likelihood                    1628.0
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      74.386
Entities:                         216   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(6,1499)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             1429.5
                            

### Conclusion

Both Entity and 2-way fixed effects estimation are significant but considering the insignificance of the time effects model, it could be inferred that the entity effects model could be preferred to the 2-way fixed effects model as it is already capable of capturing the within cluster effects of the model. Furthermore, when conducting cluster robust estimations, accounting for the entity clusters seem to achieve a robust model whereas time clusters could possibly even misspecify the model considering the inflated F-Statistic.

### Saving the models

In [19]:
save_pickle(entity_fe_res, "model/entity_fe.pickle")
save_pickle(time_fe_res, "model/time_fe.pickle")
save_pickle(tw_fe_res, "model/tw_fe.pickle")

save_pickle(entity_clus_entity_fe, "model/ind_clust_entity_fe.pickle")
save_pickle(entity_clus_time_fe, "model/ind_clust_time_fe.pickle")
save_pickle(entity_clus_tw_fe, "model/ind_clust_tw_fe.pickle")